# TimesFM Time Series Prediction Playbook

[Huggingface](https://huggingface.co/google/timesfm-2.5-200m-pytorch)

In [ ]:
def extend_path():
    """Extend notebooks system path config to import relative packages."""
    import sys
    from pathlib import Path

    parent_folder = str(Path.cwd().parent)
    print(f"Adding {parent_folder} to sys.path")
    if parent_folder not in sys.path:
        sys.path.insert(0, parent_folder)


extend_path()

In [ ]:
import torch
import timesfm

from playground.utils import plot_forecast

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

MODEL_HF_REPO_ID = "google/timesfm-2.5-200m-pytorch"
N_TIMESERIES = 1

In [ ]:
model = timesfm.TimesFM_2p5_200M_torch.from_pretrained(
    MODEL_HF_REPO_ID, torch_compile=True
)

In [ ]:
model.compile(
    timesfm.ForecastConfig(
        max_context=1024,
        max_horizon=256,
        normalize_inputs=True,
        use_continuous_quantile_head=True,
        force_flip_invariance=True,
        infer_is_positive=True,
        fix_quantile_crossing=True,
    )
)

In [ ]:
torch.manual_seed(42)
context = torch.rand((N_TIMESERIES, 128), device=DEVICE)
context.shape

In [ ]:
point_forecast, quantile_forecast = model.forecast(
    horizon=64,
    inputs=[
        context.squeeze(0),  # type: ignore[arg-type]
    ],
)

In [ ]:
point_forecast.shape, quantile_forecast.shape, quantile_forecast[0, 0, :]

> quantile dimension for timesFM: mean, then 10th to 90th quantiles.

In [ ]:
quantiles = quantile_forecast[:, :, 1:]
mean = quantile_forecast[:, :, 0]
quantiles.shape, mean.shape

In [ ]:
plot_forecast(
    ctx=context,
    quantile_fc=torch.from_numpy(quantiles),
    real_future_values=None,
    batch_idx=0,
)